In [1]:
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

In [2]:
def load_image(image_id):
    image = cv2.imread(image_id)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# train_images = train_data["image_id"].sample(100, random_state=2020).progress_apply(load_image)

In [3]:
def init_grabcut_mask(h, w):
    mask = np.ones((h, w), np.uint8) * cv2.GC_PR_BGD
    mask[h//4:3*h//4, w//4:3*w//4] = cv2.GC_PR_FGD
    mask[2*h//5:3*h//5, 2*w//5:3*w//5] = cv2.GC_FGD
    return mask

# plt.imshow(init_grabcut_mask(3*136, 3*205))

In [4]:
def add_contours(image, mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    if len(contours) != 0:
        cv2.drawContours(image, contours, -1, (255, 0, 0), 3)
        c = max(contours, key = cv2.contourArea)
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0) ,2)
    


def remove_background(image):
    h, w = image.shape[:2]
    mask = init_grabcut_mask(h, w)
    bgm = np.zeros((1, 65), np.float64)
    fgm = np.zeros((1, 65), np.float64)
    cv2.grabCut(image, mask, None, bgm, fgm, 1, cv2.GC_INIT_WITH_MASK)
    mask_binary = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
#     result = cv2.bitwise_and(image, image, mask = mask_binary)
#     add_contours(result, mask_binary) # optional, adds visualizations
    return mask_binary



In [5]:
%%time
  
import os
train = os.listdir("../input/plant-pathology-2021-fgvc8/train_images")[:10000]
for i in train:
    im = load_image("../input/plant-pathology-2021-fgvc8/train_images/"+i)
    im = cv2.resize(im, (150, 100))
    im = remove_background(im)
    cv2.imwrite(i,im)
    

CPU times: user 38min 37s, sys: 4min 57s, total: 43min 35s
Wall time: 44min 34s


In [6]:
!mkdir masks
!mv * masks

mv: cannot move 'masks' to a subdirectory of itself, 'masks/masks'


In [7]:
!apt install zip





The following NEW packages will be installed:
  zip
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 234 kB of archives.
After this operation, 623 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian stretch/main amd64 zip amd64 3.0-11+b1 [234 kB]
Fetched 234 kB in 0s (371 kB/s)
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package zip.
(Reading database ... 74146 files and directories currently installed.)
Preparing to unpack .../zip_3.0-11+b1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 16%] [#########.................................................] 8Unpacking zip (3.0-11+b1) ...
7Progress: [ 33%] [###################.......................................] 87Progress: [ 50%] [#############################.............................] 8Setting up zip (3.0-11+b1) ...
7Progress: [ 66%] [#########

In [8]:
!zip -r masks.zip masks 


  adding: masks/ (stored 0%)
  adding: masks/ff8ac2d286a6a43c.jpg (deflated 20%)
  adding: masks/93fbe07c451a874a.jpg (deflated 21%)
  adding: masks/eeb07e6a6a5a8231.jpg (deflated 19%)
  adding: masks/e8df04865c3aad95.jpg (deflated 21%)
  adding: masks/85acd23a0b659d97.jpg (deflated 18%)
  adding: masks/9361476766879c53.jpg (deflated 20%)
  adding: masks/8afa9687aab53506.jpg (deflated 20%)
  adding: masks/eb17d7a05019b58b.jpg (deflated 21%)
  adding: masks/828535ca4b7e2ed6.jpg (deflated 23%)
  adding: masks/c7b440dc95656f92.jpg (deflated 25%)
  adding: masks/f4ca866c9c7c9847.jpg (deflated 22%)
  adding: masks/f6d2c49627782ce1.jpg (deflated 21%)
  adding: masks/c4bd277cd8838a2b.jpg (deflated 24%)
  adding: masks/a564f35bdc6c5602.jpg (deflated 18%)
  adding: masks/a884dccbafe40bac.jpg (deflated 17%)
  adding: masks/e43c82e61d86e97a.jpg (deflated 19%)
  adding: masks/ca1dd4d2ef8c870a.jpg (deflated 19%)
  adding: masks/d4671a5e8d64c761.jpg (deflated 25%)
  adding: masks/a02dcfdab0b46745.jp

In [9]:
!rm masks -r